In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate

import openpyxl
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.utils.dataframe import dataframe_to_rows

def search_table(string):
    sql_tables = pd.read_sql('''
    select *
    from information_schema.tables
    ''', string_connect)

    sql_tables = sql_tables[sql_tables['TABLE_NAME'].str.contains(string)]

    def concat_values(row):
        return f"{row['TABLE_SCHEMA']}.{row['TABLE_NAME']}"

    sql_tables = sql_tables.reset_index(drop=True)

    sql_tables = sql_tables[['TABLE_NAME']]

    print(tabulate(sql_tables, headers='keys', tablefmt='pretty'))

def list_columns(table):

    sql_columns = pd.read_sql('''
    select name
    from sys.columns
    where object_id = OBJECT_ID('''+'\''+table+'\''+''');
    ''',string_connect)

    for i in range(len(sql_columns)):
        if i != len(sql_columns) - 1:
            print(sql_columns['name'][i], ', ')
        else: print(sql_columns['name'][i])

def export_to_excel(dataframes, filename):
    # Create a workbook
    workbook = openpyxl.Workbook()
    
    def get_df_name(df):
        name = [x for x in globals() if globals()[x] is df][0]
        return name

    for df in dataframes:

        # get the name of the dataframe
        sheet_name = get_df_name(df)

        # Create a worksheet
        worksheet = workbook.create_sheet(sheet_name)
        
        # Write the data from the dataframe to the worksheet
        for row in dataframe_to_rows(df, index=False, header=True):
            worksheet.append(row)
        
        # Set the column widths
        for column in df:
            column_length = max(df[column].astype(str).map(len).max(), len(column)) + 10
            col_idx = df.columns.get_loc(column)
            worksheet.column_dimensions[openpyxl.utils.get_column_letter(col_idx+1)].width = column_length

        # Freeze the header row
        worksheet.freeze_panes = worksheet['A2']
        
        # Enable filters for the worksheet
        num_rows, num_cols = df.shape
        cell_range = f'A1:{openpyxl.utils.get_column_letter(num_cols)}{num_rows}'
        worksheet.auto_filter.ref = cell_range
    
    del workbook['Sheet']
    
    # Save the workbook
    workbook.save(filename)

def login(dev_or_prod):

    if dev_or_prod == 'dev':
        user_sql = 'victorodg'
        pass_sql = '123456'
        server = 'SQLDEV001'
        database = 'DBSV001'

    elif dev_or_prod == 'prod':
        user_sql = 'victorodg'
        pass_sql = '123456'
        server = 'SQLPRC001'
        database = 'DBSV001'

    string_connect = 'mssql+pyodbc://' + user_sql + ':' + pass_sql + '@' + server + '/' + database + '?driver=SQL+Server'
    engine = create_engine(string_connect,echo=False,pool_size=50,max_overflow=50)

    return [string_connect, engine]

# Login

In [ ]:
#log = 'prod'
log = 'dev'

string_connect = login(log)[0]
engine = login(log)[1]

# List tables

In [ ]:
search_table('string_to_search_in_table_name')

# List columns

In [ ]:
list_columns('table_name')

# Query

In [ ]:
df = pd.read_sql('''

select
*

from <table>
where <argument>
order by <order> desc

''', string_connect)

# Export to .xlsx

In [ ]:
export_to_excel([df],'./files/sql_query.xlsx')